In [1]:
import pandas as pd

spotify = pd.read_csv('spotify_data.csv')

In [2]:
spotify.head()

,Unnamed: 0,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0,Jason Mraz,I Won't Give Up,53QF56cjZA9RTuuMZDrSA6,68,2012,acoustic,0.483,0.303,4,-10.058,1,0.0429,0.6940,0.000000,0.1150,0.139,133.406,240166,3
1,1,Jason Mraz,93 Million Miles,1s8tP3jP4GZcyHDsjvw218,50,2012,acoustic,0.572,0.454,3,-10.286,1,0.0258,0.4770,0.000014,0.0974,0.515,140.182,216387,4
2,2,Joshua Hyslop,Do Not Let Me Go,7BRCa8MPiyuvr2VU3O9W0F,57,2012,acoustic,0.409,0.234,3,-13.711,1,0.0323,0.3380,0.000050,0.0895,0.145,139.832,158960,4
3,3,Boyce Avenue,Fast Car,63wsZUhUZLlh1OsyrZq7sz,58,2012,acoustic,0.392,0.251,10,-9.845,1,0.0363,0.8070,0.000000,0.0797,0.508,204.961,304293,4
4,4,Andrew Belle,Sky's Still Blue,6nXIYClvJAfi6ujLiKqEq8,54,2012,acoustic,0.430,0.791,6,-5.419,0,0.0302,0.0726,0.019300,0.1100,0.217,171.864,244320,4


In [27]:
print(spotify['artist_name'].nunique())
print(spotify['genre'].nunique())
print(len(spotify))

64158
82
1159764


In [28]:
# find top 10 genre by frequency
top10_genre = spotify['genre'].value_counts().head(10).index

# filter only those genres
df_top10 = spotify[spotify['genre'].isin(top10_genre)]

# sample 5000 rows AFTER filtering
df_sample_5k = df_top10.sample(n=5000, random_state=0)

df_sample_5k.shape

(5000, 20)

In [7]:
df_sample_5k.tail()

,Unnamed: 0,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
549234,549234,Anxious,Let Me,7L77eB8vi2vpuqniEHqdzl,31,2022,emo,0.314,0.995,11,-4.836,0,0.2340,0.000165,0.000029,0.7100,0.152,186.301,161100,4
214730,214730,The Record Company,Off The Ground,3xRzUblYGZhVs52fMjii7R,54,2016,alt-rock,0.543,0.696,7,-4.572,1,0.0426,0.601000,0.215000,0.3840,0.849,92.150,253747,4
253493,253493,Mattias Krantz,Scared to be Lonely,12hmhKxOvM5fmjG6j7sUxi,23,2017,acoustic,0.496,0.358,1,-8.427,0,0.0513,0.923000,0.949000,0.1300,0.407,138.258,163840,4
74539,74539,The Smith Street Band,Don't Fuck with Our Dreams,30JdRSCH8S2e1byMgpthEI,14,2013,emo,0.339,0.783,4,-4.300,1,0.0619,0.014500,0.000005,0.7370,0.479,146.841,326264,4
181916,181916,Deitrick Haddon,Be Like Jesus,5MnWstVq8yVUpBSKiBTyTA,27,2015,gospel,0.530,0.633,1,-5.915,1,0.3490,0.016400,0.000000,0.0631,0.432,201.693,199565,4


In [29]:
import altair as alt

brush = alt.selection_interval(encodings=['x','y'])  # rectangular brush on scatter

input_dropdown = alt.binding_select(options = [None] + sorted(df_sample_5k['genre'].unique()),
                                    labels = ['All'] + sorted(df_sample_5k['genre'].unique()),
                                   name = 'Genre: ')

selection = alt.selection_point(fields = ['genre'], bind = input_dropdown)

scatter = (
    alt.Chart(df_sample_5k)
    .mark_circle(opacity=0.6)
    .encode(
        x='energy:Q', y='popularity:Q', 
        color=alt.condition(selection, 'genre:N', alt.value('lightgray')),
        opacity = alt.condition(selection, alt.value(1), alt.value(0.1))
    )
    .add_params(brush, selection)
    .properties(width=300, height=220)
    #.interactive()  # pan/zoom on continuous scales
)

hist = (
    alt.Chart(df_sample_5k)
    .transform_filter(selection)
    .transform_filter(brush)
    .mark_bar()
    .encode(
        x=alt.X('year:O', title='Year'),
        y=alt.Y('count()', title='Count'),
        color='genre:N',
        tooltip=['count():Q','year:T','artist_name:N']
    )
    .properties(width=300, height=220)
)

fig1 = (scatter | hist)

fig1.save('example1.html')

c:\Users\yangx\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
c:\Users\yangx\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


In [30]:
time_brush = alt.selection_interval(encodings=['x'])

overview = (
    alt.Chart(df_sample_5k)
    .mark_area(opacity=0.4)
    .encode(
        x=alt.X('year:O', title=''),
        y=alt.Y('mean(popularity):Q', title='Mean popularity'),
        color='genre:N'
    )
    .properties(height=80, width=620)
    .add_params(time_brush)
)

detail = (
    alt.Chart(df_sample_5k)
    .transform_filter(time_brush)
    .mark_line()
    .encode(
        x=alt.X('year:O', title='Time'),
        y=alt.Y('mean(popularity):Q', title='Mean Popularity'),
        color='genre:N',
        tooltip=['time:T','mean(popularity):Q','singer:N']
    )
    .properties(height=180, width=620)
)

fig2 = overview & detail

fig2.save('example2.html')


c:\Users\yangx\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
c:\Users\yangx\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


In [12]:
top10 = spotify['genre'].value_counts().head(12).index
df_top = spotify[spotify['genre'].isin(top10)]

In [13]:
len(df_top)

246615

In [14]:
genre_year_matrix = df_top.pivot_table(
    index='year',
    columns='genre',
    aggfunc='size',
    fill_value=0
)

In [15]:
genre_year_matrix = genre_year_matrix.drop('ambient', axis = 1)
genre_year_matrix = genre_year_matrix.drop('forro', axis = 1)
genre_year_matrix

genre,acoustic,alt-rock,black-metal,blues,comedy,emo,gospel,indian,k-pop,new-age
year,,,,,,,,,,
2000,841,896,874,782,289,854,947,874,927,940
2001,673,842,915,838,377,861,942,853,835,854
2002,826,882,884,848,407,871,921,878,832,834
2003,869,842,849,744,480,928,863,820,821,801
2004,846,797,883,804,473,895,886,819,857,790
2005,841,871,839,785,781,905,848,756,805,862
2006,847,810,876,775,898,818,904,936,821,795
2007,856,907,918,770,924,804,871,820,840,812
2008,806,898,880,797,879,809,881,806,780,837


In [16]:
genre_year_matrix.to_csv('genre_year_matrix.csv')